In [155]:
import numpy as N, pylab as M
import ipyvolume as ipv

In [158]:
## Make a random, spherically symmetric point cloud for nodes
rmax = 100
np=3000
rs = N.random.RandomState(234) #Random seed
nlink=200

# profileindex controls the distribution of radii.
# Set it to 1./3. for a volumetrically uniform profile 

profileindex=1./3.
r=rmax * (N.arange(np)/float(np))**profileindex

##Fully random
costheta=1.-2.*rs.rand(np)
phi = 2.*N.pi*rs.rand(np)

##For maximum uniformity
#step = 1./float(np+1)
#costheta = 1.-2.*N.arange(step/2., 1.-step/2.,step)
#phi=2.*N.pi*N.arange(step/2., 1.-step/2.,step)
#costheta = costheta[rs.permutation(np)]
#phi = phi[rs.permutation(np)]

sintheta = N.sqrt(1.-costheta**2)

xyz = N.empty((np,3),dtype=N.float32)
xyz[:,0] = r*sintheta*N.cos(phi)
xyz[:,1] = r*sintheta*N.sin(phi)
xyz[:,2] = r*costheta

In [159]:
# Show random node set
fig = ipv.figure(width=400, height=400)
ipv.scatter(xyz[:1000,0],xyz[:1000,1],xyz[:1000,2],marker='sphere',size=1)
ipv.show()

In [160]:
# Grow neuron grafted onto nodes

#Does the neuron include node i?
inneuron=N.zeros(np,dtype=N.bool)
#The soma point is in the neuron
inneuron[0]=True
#What is a node's parent?
linkedto= N.zeros(np,dtype=N.int) 
#What is the distance within the neuron to the node, through its parent nodes?
#(Necessarily the shortest distance, since nodes don't have multiple parents)
distinneuron = N.zeros(np,dtype=N.float)
#Branching parameter: main "One Rule to Make them All" parameter
bf=0.2
for link in N.arange(nlink):
    # Look only for nodes to add not currently in the neuron. 
    # nodes don't have multiple parents! -- not quite like the cosmic web
    wout = N.where(N.logical_not(inneuron))[0]

    #Loop through particles within the neuron
    win = N.where(inneuron)[0]
    for iwin in range(len(win)):
        dist2neuroni=N.sqrt(N.sum((xyz[wout,:]-xyz[win[iwin],:])**2,axis=1))+ bf*distinneuron[win[iwin]]
        if iwin == 0:
            dist2neuron=N.copy(dist2neuroni)
        else:
            dist2neuron = N.minimum(dist2neuron,dist2neuroni)
    # dist2neuron is the minimum distance from each node outside the neuron
    # to any node inside the neuron
    wherelink = N.argmin(dist2neuron)
    addedpoint = wout[wherelink]
    inneuron[addedpoint] = True
    #figure out which parent it should be attached to
    # (same computation as inside the loop, but now the array is over possible parent nodes
    dist2added = N.sqrt(N.sum((xyz[win,:]-xyz[wout[wherelink],:])**2,axis=1)) + bf*distinneuron[win]
    # assign the parent
    linkedto[addedpoint] = win[N.argmin(dist2added)]
    # Already have distinneuron for the parent, so we can just add the dist to the parent
    # to get distinneuron for the child
    distinneuron[addedpoint] = dist2neuron[wherelink] + distinneuron[linkedto[addedpoint]]

In [162]:
# Display lines
win = N.where(inneuron)[0]
fig = ipv.figure(width=400, height=400)

for i in N.arange(1,len(win)): 
    # don't start at 0 since we're plotting lines from a node to its parent
    wi = win[i]
    lineids = [wi,linkedto[wi]]
    ipv.plot(xyz[lineids,0],xyz[lineids,1],xyz[lineids,2])
    
ipv.show()